<a href="https://colab.research.google.com/github/girija2204/DLTopics/blob/master/TFRecordSetupForOlivetti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
CUT_FACE_IMG_LENGTH = 64
CUT_FACE_IMG_BREADTH = 64

In [ ]:
ol_faces = datasets.fetch_olivetti_faces()

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def _float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [ ]:
def convert_toexample(row):
    feature_dict = {}
    image_tensor, label = row
    feature_dict['image'] = _float_list_feature(image_tensor)
    feature_dict['label'] = _int64_feature(label)
    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example

In [ ]:
def convert_to_tfrecord(images: np.ndarray, labels: np.ndarray):
    writer = tf.io.TFRecordWriter(get_tfrecords_file())
    write_count = 0
    for i in range(0, images.shape[0]):
        image = np.reshape(images[i],
                           [CUT_FACE_IMG_LENGTH * CUT_FACE_IMG_BREADTH, ])
        row = (image, labels[i])
        example = convert_toexample(row)
        writer.write(example.SerializeToString())
        write_count = write_count + 1
    print(f"Number of images {write_count}, saved into location: {get_tfrecords_file()} as tfrecords.")

In [ ]:
def get_data(dataset, count=None, same_label=False, label=None, categories=None):
    images = dataset.images
    targets = dataset.target

    if count is None:
        return images, targets

    if same_label:
        if label is None:
            return "Label not provided.., It's a mandatory parameter if same_label is set to True"
        all_imgs_of_label = images[targets == label]
        if len(all_imgs_of_label) >= count:
            out_data = all_imgs_of_label[:count]
            out_targets = np.array([label] * count, np.uint8)
        else:
            out_data = all_imgs_of_label
            out_targets = np.array([label] * all_imgs_of_label.shape[0], np.uint8)
    else:
        label_wise_imgs = np.zeros(shape=(count, images[0].shape[0], images[0].shape[1]))
        label_wise_tgts = np.zeros(shape=count)
        for index, category in enumerate(categories):
            if index == count:
                break
            else:
                label_wise_imgs[index] = images[targets == category][0]
                label_wise_tgts[index] = category
        out_data = label_wise_imgs
        out_targets = label_wise_tgts
    return out_data, out_targets

In [ ]:
ol_images, ol_targets = get_data(dataset=ol_faces)